# HashiCorp Vault Demo Setup 

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup. The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your `VAULT_ADDR` and `VAULT_TOKEN` variables.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin. To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Vault

In [1]:
# For this demo, we will be passing our doormat credentials as environment variables.
# For non-doormat scenarios, use your configured IAM programmatic credentials in the secret sync configuration later on.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/Documents/Misc/vaultControlGroup/vault.hclic)

# Refresh Vault docker image with latest version
docker pull hashicorp/vault-enterprise

# Run Vault in docker in Dev mode with Enterprise license.
# We have set VAULT_LOG_LEVEL to trace for troubleshooting purposes.  This will allow you to view detailed information as you test.
# docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
# -e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
# -e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
# -e "VAULT_LICENSE=${VAULT_LICENSE}" \
# -e "VAULT_LOG_LEVEL=trace" \
# -p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Pinning to 1.15-ent as 1.16.1 now does not support AWS session tokens for secrets 
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=trace" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:1.15-ent


Using default tag: latest
latest: Pulling from hashicorp/vault-enterprise

c3f34fad: Pulling fs layer 
9f20aea6: Pulling fs layer 
6dfde488: Pulling fs layer 
c3ec412c: Pulling fs layer 
33a9e127: Pulling fs layer 
3ccc0c20: Pulling fs layer 
Digest: sha256:c7e266b84e44580d1f5496c91c23e051f96c75a8774de4b413a563119768c50f
Status: Downloaded newer image for hashicorp/vault-enterprise:latest
docker.io/hashicorp/vault-enterprise:latest
Unable to find image 'hashicorp/vault-enterprise:1.15-ent' locally
1.15-ent: Pulling from hashicorp/vault-enterprise

c3f34fad: Already exists 
94bae40f: Pulling fs layer 
96a95acb: Pulling fs layer 
4bdc191d: Pulling fs layer 
22efbf31: Pulling fs layer 
9527950f: Pulling fs layer 
Digest: sha256:c7f1f4d2bc2fe0bab903bb46b699d28c839149b5c549af028d04c83e100f4f39
Status: Downloaded newer image for hashicorp/vault-enterprise:1.15-ent
141bb731fe387031515ed946415d86d9ee13fa9c6b4b43748146689738ec7150


In [2]:
# Verify that Vault is running
docker ps
echo
vault status

CONTAINER ID   IMAGE                                 COMMAND                  CREATED         STATUS         PORTS                                       NAMES
141bb731fe38   hashicorp/vault-enterprise:1.15-ent   "docker-entrypoint.s…"   8 seconds ago   Up 8 seconds   0.0.0.0:8200->8200/tcp, :::8200->8200/tcp   vault-enterprise

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.15.14+ent
Build Date      2024-08-26T17:17:05Z
Storage Type    inmem
Cluster Name    vault-cluster-80a83620
Cluster ID      205e11fd-8453-b62c-c5fb-71151e4ab05e
HA Enabled      false


In [3]:
# Verify license features
vault read sys/license/status -format=json

{
  "request_id": "8ef0c4b4-ab6d-0519-1fef-a3940389b8c3",
  "lease_id": "",
  "lease_duration": 0,
  "renewable": false,
  "data": {
    "autoloaded": {
      "expiration_time": "2024-09-06T00:00:00Z",
      "features": [
        "HSM",
        "Performance Replication",
        "DR Replication",
        "MFA",
        "Sentinel",
        "Seal Wrapping",
        "Control Groups",
        "Performance Standby",
        "Namespaces",
        "KMIP",
        "Entropy Augmentation",
        "Transform Secrets Engine",
        "Lease Count Quotas",
        "Key Management Secrets Engine",
        "Automated Snapshots",
        "Key Management Transparent Data Encryption",
        "Secrets Sync"
      ],
      "license_id": "ca2df2a4-d27d-7573-3a77-10ad4387d4f3",
      "performance_standby_count": 9999,
      "start_time": "2024-08-29T00:00:00Z",
      "termination_time": "2024-09-06T00:00:00Z"
    },
    "autoloading_used": true,
    "persisted_autoload": {
      "expiration_time": "2024-0

# Clean up

In [11]:
# Stop Vault container
docker stop vault-enterprise

vault-enterprise
